In [1]:
# https://platform.olimpiada-ai.ro/problems/47

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import torch

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [3]:
train = pd.read_csv("/kaggle/input/identifying-languages/train.csv")
test = pd.read_csv("/kaggle/input/identifying-languages/test.csv")

In [4]:
from sklearn.model_selection import train_test_split

train, _ = train_test_split(train, random_state=42, train_size=1000, stratify=train['language'])

In [5]:
class2idx = {}
for i, lang in enumerate(train['language'].unique()):
    class2idx[lang] = i

idx2class = {v: k for k, v in class2idx.items()}

train['language'] = train['language'].map(class2idx.get)
train.rename(columns={'language': 'labels'}, inplace=True)

In [6]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained('intfloat/multilingual-e5-large')
model = AutoModelForSequenceClassification.from_pretrained('intfloat/multilingual-e5-large', num_labels=len(class2idx)).to(device)

tokenizer_config.json:   0%|          | 0.00/418 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/690 [00:00<?, ?B/s]

...

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at intfloat/multilingual-e5-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
import torch

input_texts = ['query: how much protein should a female eat',
               'query: 南瓜的家常做法',
               "passage: As a general guideline, the CDC's average requirement of protein for women ages 19 to 70 is 46 grams per day. But, as you can see from this chart, you'll need to increase that if you're expecting or training for a marathon. Check out the chart below to see how much protein you should be eating each day.",
               "passage: 1.清炒南瓜丝 原料:嫩南瓜半个 调料:葱、盐、白糖、鸡精 做法: 1、南瓜用刀薄薄的削去表面一层皮,用勺子刮去瓤 2、擦成细丝(没有擦菜板就用刀慢慢切成细丝) 3、锅烧热放油,入葱花煸出香味 4、入南瓜丝快速翻炒一分钟左右,放盐、一点白糖和鸡精调味出锅 2.香葱炒南瓜 原料:南瓜1只 调料:香葱、蒜末、橄榄油、盐 做法: 1、将南瓜去皮,切成片 2、油锅8成热后,将蒜末放入爆香 3、爆香后,将南瓜片放入,翻炒 4、在翻炒的同时,可以不时地往锅里加水,但不要太多 5、放入盐,炒匀 6、南瓜差不多软和绵了之后,就可以关火 7、撒入香葱,即可出锅"]


batch_dict = tokenizer(input_texts, max_length=512, padding=True, truncation=True, return_tensors='pt')
batch_dict = {k: v.to(device) for k, v in batch_dict.items()}

logits = model(**batch_dict).logits
preds = [idx2class[v] for v in torch.argmax(torch.softmax(logits, dim=1), dim=1).tolist()]
preds

['Indonesian', 'Urdu', 'Indonesian', 'Urdu']

In [8]:
from datasets import Dataset

train_ds = Dataset.from_pandas(train)
test_ds = Dataset.from_pandas(test)

train_ds = train_ds.train_test_split(seed=42, test_size=0.1)

In [9]:
def tokenize_fn(examples):
    return tokenizer(examples['Text'], max_length=512, padding=True, truncation=True, return_tensors='pt')

train_ds = train_ds.map(tokenize_fn, batched=True, remove_columns=['SampleID', 'Text'])
test_ds = test_ds.map(tokenize_fn, batched=True, remove_columns=['SampleID', 'Text'])

Map:   0%|          | 0/900 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/4400 [00:00<?, ? examples/s]

In [10]:
from transformers import TrainingArguments, Trainer

args = TrainingArguments(
    output_dir='./result',
    report_to='none',
    do_train=True,
    num_train_epochs=1,
    learning_rate=2e-5,
    weight_decay=1e-3,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    do_eval=True,
    eval_strategy='steps',
    eval_steps=0.1,
    save_strategy='steps',
    seed=42
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_ds['train'],
    eval_dataset=train_ds['test'],
    processing_class=tokenizer,
)

In [11]:
trainer.train()

/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Step,Training Loss,Validation Loss
45,No log,3.125873
90,No log,2.882288
135,No log,2.071462
180,No log,1.132932
225,No log,0.614620
270,No log,0.289664
315,No log,0.187037
360,No log,0.150749
405,No log,0.145337
450,No log,0.132971


TrainOutput(global_step=450, training_loss=1.3298848470052083, metrics={'train_runtime': 600.0392, 'train_samples_per_second': 1.5, 'train_steps_per_second': 0.75, 'total_flos': 769987510812000.0, 'train_loss': 1.3298848470052083, 'epoch': 1.0})

In [15]:
predictions = trainer.predict(test_ds)

In [20]:
preds = np.argmax(predictions.predictions, axis=1)
preds

array([ 5,  2, 10, ...,  0,  8, 18])

In [23]:
subm = pd.DataFrame({
    'SampleID': test['SampleID'],
    'language': list(map(idx2class.get, preds))
})

subm

,SampleID,language
0,S17601,Japanese
1,S17602,Russian
2,S17603,Latin
3,S17604,Pushto
4,S17605,Hindi
...,...,...
4395,S21996,Dutch
4396,S21997,Chinese
4397,S21998,English
4398,S21999,Pushto


In [24]:
subm.to_csv("submission.csv", index=False)